Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
# add the 'Bike Stop #' 

bikes_df['Bike Stop #'] = bikes_df.index

# merge the df

merged_df = final_foursquare_df.merge(bikes_df, on='Bike Stop #')
merged_df

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

I used a pair plot as a inital point of contact during the EDA process. The resulting pair plot can be viewed in files: images > EDA_pairplot.png. There are obvious strong correlations between 'free_bikes', 'empty_slots' and '% used' since they are all related to one another. This however, is not unhelpful information as it confirms that the data extraction process was successful in this limited capacity. The more interesting of the preliminary observations can be seen with the relationship between 'venues_nearby' with all other metrics. 

This will be at the core of the regression modelling that will be addressed later and may prove some interesting patterns. At a glance there seems to be a slight positive trend between 'free_bikes' + 'venue_nearby' and a slight negative trend between 'empty_slots' + 'venues_nearby'.

This qualitatively makes sense as the dataset was pulled around 11pm Barcelona local time. This relationship may show that during the evening, less bikes are used around these venues. These venues tend to be large with much commercial zoning surrounding them - leading to a general decreased use of the bikes during that hour in those zones.

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect('your_database.db')
cursor = con.cursor()

In [ ]:
#bikes to SQL

bikes_df.to_sql('bikes_df', con, index=False, if_exists='replace')

In [ ]:
# get rid of list to convert to SQL
#foursquare to SQL

final_foursquare_df['venue_type'] = final_foursquare_df['venue_type'].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
final_foursquare_df.to_sql('final_foursquare_table', conn, index=False, if_exists='replace')

In [ ]:
# get rid of list to convert to SQL
#merged to SQL

merged_df['venue_type'] = merged_df['venue_type'].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
merged_df.to_sql('merged_table', conn, index=False, if_exists='replace')

Look at the data before and after the join to validate your data.

In [ ]:
#example queries

query = 'SELECT * FROM bikes_table'  
result = pd.read_sql_query(query, conn)

In [ ]:
#example queries

query = '''
    SELECT * 
    FROM bikes_table
    JOIN final_foursquare_table
    ON bikes_table.index = finalfoursquare_table.'Bike Stop #'
    WHERE bikes_table.name IS NOT NULL
'''  
result = pd.read_sql_query(query, conn)

tables were join in SQL and compared to the original df in python. The total number of rows, total number of non-null rows, and some spot checks all matched. The data is validated as being reliable.